In [ ]:
import io
from hashlib import sha1
from tqdm.auto import tqdm, trange

from pyecsca.ec.params import DomainParameters, get_params, load_params_ectester
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.mod import mod, Mod
from pyecsca.ec.point import Point
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.key_agreement import ECDH as ECDHBase
from pyecsca.misc.utils import TaskExecutor

In [ ]:
params = load_params_ectester(io.BytesIO(b"0xa9fa3419aca88bade2cba14e317816c79d52481d463dc9bcb12c37f45aa3b4e1,0x2ea3bfe6659f8e035735349b91fbfa2baf0cf8e640315f0fe03c1136813dec99,0x2b07c518e04b02158651e3dbbef7720015dd496bf15af02f8439f8e1503b8370,0x16a1e99de6f5ac372c1565a232f9bd3fc602c70d8edb6a30e604f391750ad7ee,0x2e311109d9f55a54977e288bf19a7411353e80b2d4c08c4f80a87105889d4a05,0xa9fa3419aca88bade2cba14e317816c6828910c6ce04fcd2a2e857d66aa4d6d3,0x01"), "projective")

In [ ]:
model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
affine = AffineCoordinateModel(model)
add = coords.formulas["add-2015-rcb"]
dbl = coords.formulas["dbl-2015-rcb"]
mult = LTRMultiplier(add, dbl)

In [ ]:
priv = 0x2bc345cc91f6b17ca180beb2b5de274e1a384e9dc2f40de02eae4141a91f39d5
pub = Point(affine,
            x=mod(0x90fb04b1af19e8e20396ac052f260a9fb5f736b97e3cd4af08fe81a1e75dac6d, params.curve.prime),
            y=mod(0x2302bcf700d3d5899f04d0c7441f5017c9758bfafd6ce15dbe36fb4eea76baec, params.curve.prime))
secret = bytes.fromhex("72f7a06b7d934a9370e66f37e3745c206589d205")

In [ ]:
base = params.curve.affine_multiply(pub, priv)
pub92 = params.curve.affine_multiply(pub, 92)

In [ ]:
class ECDH(ECDHBase):
    def __init__(self, params: DomainParameters, pubkey: Point, privkey: Mod,
                 base: Point, i: int, pub92: Point):
        self.hash_algo = sha1
        self.params = params
        self.pubkey = pubkey
        self.privkey = privkey
        self.base = base
        self.i = i
        self.pub92 = pub92
        self.point = params.curve.affine_add(base, params.curve.affine_multiply(pub92, i))

    def advance(self, by: int = 1):
        if by == 1:
            self.point = params.curve.affine_add(self.point, pub92)
        else:
            multiple = params.curve.affine_multiply(pub92, by)
            self.point = params.curve.affine_add(self.point, multiple)
        self.i += by
    
    def perform_raw(self) -> Point:
        return self.point

In [ ]:
def split_ranges(bound, splits):
    """
    Splits the range [0, bound) into `splits` equally sized ranges.

    Returns:
        List of (start, end) tuples, where each tuple represents a range [start, end)
    """
    ranges = []
    chunk_size = bound // splits
    remainder = bound % splits
    start = 0
    for i in range(splits):
        end = start + chunk_size + (1 if i < remainder else 0)
        ranges.append((start, end))
        start = end
    return ranges

In [ ]:
def process_split(split, params, pub, priv, base, pub92, secret):
    start, end = split
    e = ECDH(params, pub, mod(priv, params.order), base, start, pub92)
    for i in range(*split):
        out = e.perform()
        if out == secret:
            break
        e.advance()
    else:
        return None
    return e.i

In [ ]:
num_workers = 25
splits = 10000
bound = 2**32
other_secret = bytes.fromhex("c512b8387fb403241d0a6925ff76a2c476038532")
with TaskExecutor(max_workers=num_workers) as pool:
    for split in tqdm(split_ranges(bound, splits), desc="Submitting."):
        pool.submit_task(split,
                         process_split,
                         split, params, pub, priv, base, pub92, other_secret)
    for split, future in tqdm(pool.as_completed(), desc="Computing secrets.", total=len(pool.tasks)):
        if error := future.exception():
            print("Error!", error)
            continue
        res = future.result()
        if res is not None:
            print(split)
            print(f"Found {rest}")
            break

In [ ]:
res

In [ ]:
res is None

In [ ]:
split

In [ ]:
future